In [130]:
import pandas as pd 
import numpy as np
ingr_ns=pd.read_csv('tabellen/data_ingr.csv')
ingr_schoon=pd.read_csv('tabellen/data_ingr_clean.csv') 
recipes = pd.read_csv('tabellen/Recepten_kiezen_g.csv') 

In [131]:
import csv
with open('tabellen/Recepten_kiezen_f.csv', newline='') as csvfile:
    csvlezer = csv.reader(csvfile, delimiter=',', quotechar='"')
    for x in csvlezer:
        x = recipes
recipes[117:123]

recipes

,Unnamed: 0,title,persons,time,calories,stars,url,image,Unnamed: 8
0,Kika:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 wel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Courgette carbonara,4.0,20.0,575.0,0.0,https://www.ah.nl/allerhande/recept/R-R1188954...,https://static.ah.nl/static/recepten/img_RAM_P...,NaN
3,0,Mexicaanse taco's met vis en zoete aardappel,4.0,20.0,655.0,0.0,https://www.ah.nl/allerhande/recept/R-R1188828...,https://static.ah.nl/static/recepten/img_09292...,NaN
4,0,Gegrilde-paprikasalade met feta en munt,4.0,15.0,205.0,0.0,https://www.ah.nl/allerhande/recept/R-R1188715...,https://static.ah.nl/static/recepten/img_09094...,NaN
...,...,...,...,...,...,...,...,...,...
131,0,Rodekoolsteak met blauwaderkaas en truffel,4.0,15.0,550.0,0.0,https://www.ah.nl/allerhande/recept/R-R1189726...,https://static.ah.nl/static/recepten/img_10034...,NaN
132,0,Vegan wraps met tofu en rodekoolsalade,4.0,20.0,670.0,0.0,https://www.ah.nl/allerhande/recept/R-R1195491...,https://static.ah.nl/static/recepten/img_RAM_P...,NaN
133,0,Ovenschotel met hachee rodekool en puree,4.0,30.0,630.0,0.0,https://www.ah.nl/allerhande/recept/R-R1192021...,https://static.ah.nl/static/recepten/img_12333...,NaN
134,0,Aardappel en zuurkool,4.0,15.0,405.0,0.0,https://www.ah.nl/allerhande/recept/R-R758540/...,https://static.ah.nl/static/recepten/img_00767...,NaN


In [132]:
ingr

,Unnamed: 0,recipe,ingredient,quantity,unit
0,0,Kruidnoten met choco-discodip,melkchocolade,100.0,g
1,1,Kruidnoten met choco-discodip,kruidnoten,100.0,g
2,2,Kruidnoten met choco-discodip,discodip,2.0,el
3,3,Kruidnoten in marsepein,blanke marsepein,150.0,g
4,4,Kruidnoten in marsepein,ongezouten roomboter,15.0,g
...,...,...,...,...,...
49814,71804,Biefstuk met rodewijnsaus en ham,olijfolie,4.0,el
49815,71805,Biefstuk met rodewijnsaus en ham,biefstukken,4.0,NaN
49816,71806,Biefstuk met rodewijnsaus en ham,boter,25.0,g
49817,71807,Biefstuk met rodewijnsaus en ham,serranoham,4.0,plakken


In [133]:
onze_ns = ingr_ns[ingr_ns['recipe'].isin(recipes['title'])]

In [134]:
onze_schoon = ingr_schoon[ingr_schoon['recipe'].isin(recipes['title'])]

In [135]:
class Recepten: 
    def __init__(self, df): 
        self.data_frame = df
        

In [136]:
recept_schoon = Recepten(ingr_schoon)

print(recepten.data_frame)

       Unnamed: 0                            recipe            ingredient  \
0               0     Kruidnoten met choco-discodip         melkchocolade   
1               1     Kruidnoten met choco-discodip            kruidnoten   
2               2     Kruidnoten met choco-discodip              discodip   
3               3           Kruidnoten in marsepein      blanke marsepein   
4               4           Kruidnoten in marsepein  ongezouten roomboter   
...           ...                               ...                   ...   
49814       71804  Biefstuk met rodewijnsaus en ham             olijfolie   
49815       71805  Biefstuk met rodewijnsaus en ham           biefstukken   
49816       71806  Biefstuk met rodewijnsaus en ham                 boter   
49817       71807  Biefstuk met rodewijnsaus en ham            serranoham   
49818       71808  Biefstuk met rodewijnsaus en ham             rode wijn   

       quantity     unit  
0         100.0        g  
1         100.0      

In [137]:
recept_ns = Recepten(ingr_ns)

print(recepten.data_frame)

       Unnamed: 0                            recipe            ingredient  \
0               0     Kruidnoten met choco-discodip         melkchocolade   
1               1     Kruidnoten met choco-discodip            kruidnoten   
2               2     Kruidnoten met choco-discodip              discodip   
3               3           Kruidnoten in marsepein      blanke marsepein   
4               4           Kruidnoten in marsepein  ongezouten roomboter   
...           ...                               ...                   ...   
49814       71804  Biefstuk met rodewijnsaus en ham             olijfolie   
49815       71805  Biefstuk met rodewijnsaus en ham           biefstukken   
49816       71806  Biefstuk met rodewijnsaus en ham                 boter   
49817       71807  Biefstuk met rodewijnsaus en ham            serranoham   
49818       71808  Biefstuk met rodewijnsaus en ham             rode wijn   

       quantity     unit  
0         100.0        g  
1         100.0      

In [152]:
recipes_schoon = recipes.dropna(subset=['title'])
recipes_schoon

,Unnamed: 0,title,persons,time,calories,stars,url,image,Unnamed: 8
2,0,Courgette carbonara,4.0,20.0,575.0,0.0,https://www.ah.nl/allerhande/recept/R-R1188954...,https://static.ah.nl/static/recepten/img_RAM_P...,NaN
3,0,Mexicaanse taco's met vis en zoete aardappel,4.0,20.0,655.0,0.0,https://www.ah.nl/allerhande/recept/R-R1188828...,https://static.ah.nl/static/recepten/img_09292...,NaN
4,0,Gegrilde-paprikasalade met feta en munt,4.0,15.0,205.0,0.0,https://www.ah.nl/allerhande/recept/R-R1188715...,https://static.ah.nl/static/recepten/img_09094...,NaN
5,0,Viscurry met zelfgemaakte currypasta,4.0,20.0,345.0,0.0,https://www.ah.nl/allerhande/recept/R-R1187967...,https://static.ah.nl/static/recepten/img_08598...,NaN
6,0,Spruitjesgratin met zoete aardappel,4.0,10.0,665.0,0.0,https://www.ah.nl/allerhande/recept/R-R1187741...,https://static.ah.nl/static/recepten/img_08516...,NaN
...,...,...,...,...,...,...,...,...,...
131,0,Rodekoolsteak met blauwaderkaas en truffel,4.0,15.0,550.0,0.0,https://www.ah.nl/allerhande/recept/R-R1189726...,https://static.ah.nl/static/recepten/img_10034...,NaN
132,0,Vegan wraps met tofu en rodekoolsalade,4.0,20.0,670.0,0.0,https://www.ah.nl/allerhande/recept/R-R1195491...,https://static.ah.nl/static/recepten/img_RAM_P...,NaN
133,0,Ovenschotel met hachee rodekool en puree,4.0,30.0,630.0,0.0,https://www.ah.nl/allerhande/recept/R-R1192021...,https://static.ah.nl/static/recepten/img_12333...,NaN
134,0,Aardappel en zuurkool,4.0,15.0,405.0,0.0,https://www.ah.nl/allerhande/recept/R-R758540/...,https://static.ah.nl/static/recepten/img_00767...,NaN


In [153]:
a= np.array([recipes_schoon['title']]) 
a 

array([['Courgette carbonara',
        "Mexicaanse taco's met vis en zoete aardappel",
        'Gegrilde-paprikasalade met feta en munt',
        'Viscurry met zelfgemaakte currypasta',
        'Spruitjesgratin met zoete aardappel',
        'Geroosterde groenten met gorgonzola',
        'Thaise kokosnoedelsoep met garnalen',
        "Gevulde paprika's met rijst",
        'Kikkererwtencurry met pinda en spinazie en rijst ',
        'JalapeâˆšÂ±o-poppers',
        'Geroosterde aspergebroccoli met zwarte-knoflookdip',
        'Parelcouscoussalade met broccoli en rivierkreeftjes',
        'Bagel gerookte zalm roomkaas en kappertjes',
        'Bietencarpaccio met geitenkaasdressing',
        'Miso dressing met sojasaus',
        'Pindasoep met noedels en champignons', 'Macarons',
        'Open dumplings met garnalen in Siu Mai-stijl', 'Kaaskroketten',
        'Pappardelle met asperges en zalm', 'Meloengranita',
        'Quinoasalade met linzen',
        'Bloemkool-pastinaakstamppot met spin